__No te olvides de rellenar esto:__

- Número de grupo: 11 
- Nombre de los integrantes del grupo: Arturo Ibáñez Martínez, Jorge del Valle Vázquez

# Práctica 1

> __Fecha de entrega: 11 de abril de 2021__


## Parte 2: similitud semántica

Una de las grandes ventajas de las representaciones estructuradas es que podemos aprovechar su estructura para calcular similitudes semánticas entre las entidades. En esta ocasión vamos a cacular la similitud entre dos conceptos como:

$$Sim(A, B) = \frac{\delta(root, C)}{\delta(root, C) + \delta(C, A) + \delta(C, B)}$$

siendo:

- $\delta(X, Y)$ el __mínimo__ número de aristas que conecta A y B, siendo A más general que B.
- $C = LCS(A, B)$ el concepto más específico de la jerarquía que es más general que A y B (_least common subsummer_).

La idea tras esta similitud queda reflejada en la siguiente imagen:

<img src="sim.png" alt="Similitud" style="width: 300px;"/>

En la práctica pueden existir distintos conceptos C que cumplen la definición de _least common subsummer_ de A y B por lo que es necesario definir cuál de ellos vamos a utilizar. En nuestro caso seleccionaremos __uno de los que maximiza el valor de similitud__. 

### 1) Obtener la taxonomía con la que vamos a trabajar

Utiliza el [punto el acceso](https://query.wikidata.org/) SPARQL de Wikidata para ejecutar una consulta que devuelva todos los pares de entidades $(x, y)$ tal que $x$ es subconcepto directo de $y$ y ambos son un tipos de [instrumentos musicales (Q34379)](https://www.wikidata.org/wiki/Q34379). Debes recuperar tantos las URIs de la entidades como sus etiquetas.

Escribe en la siguiente celda la consulta que has utilizado comentada adecuadamente.

A continuación descarga todas las respuestas en formato _Archivo JSON_ y guardalo en el mismo directorio de la práctica.

_Nota: en el momento de realizar esta práctica obtuve 4727 resultados pero el número puede variar al ser Wikidata una base de conocimiento dinámica._

### 2) Cargar la taxonomía en memoria

Vamos a cargar la taxonomía de clases en memoria para poder operar con ella. Representaremos la jerarquía de lcases mediantes las siguientes estructuras:

- Un diccionario que asocia a cada identificador su etiqueta (por ejemplo 'Q34379' -> 'musical instrument')
- Un diccionario que asocia cada clase con sus subclases directas (por ejemplo 'Q17172850' -> {'Q101436564', 'Q1067089', 'Q186506', 'Q210970', 'Q223166', ...}, )
- Un diccionario que asocia cada clase con sus superclases directas (por ejemplo 'Q5994' -> {'Q3152898', 'Q4951628', 'Q52954'})

Tienes libertad para elegir cómo quieres representar la taxonomía en Python:

- Puedes usar una clase. En ese caso tendrás que ir añadiendo métodos a la clase para completar cada uno de los apartados de la práctica. Escribe el código de la clase en una única celda y utiliza los métodos que necesites en cada uno de los apartados.
- Puedes usar 3 variables globales para representar la taxonomía. En ese caso deberás escribir las operaciones como funciones en cada uno de los apartados de la práctica.

En cualquier caso recuerda documentar adecuadamente el código y trata de que sea sencillo de entender.

Crea una operación _load_ que reciba el nombre del fichero json y cargue el grafo en memoria usando las estructuras anteriores.

```python
import json

with open(filename, encoding='utf-8') as f:
    data = json.load(f)
```

In [1]:
import json

with open("query.json", encoding='utf-8') as f:
    data = json.load(f)

In [6]:
## Ejemplos para entender el acceso del json
import json
jsonData = '[{"name": "Frank", "age": 39},{"name": "Jorge", "age": 21}]'
jsonToPython = json.loads(jsonData)
print (jsonToPython)
print (data [1] )
print (data [1] ['a'])
print (data [1] ['a'][31:])

[{'name': 'Frank', 'age': 39}, {'name': 'Jorge', 'age': 21}]
{'a': 'http://www.wikidata.org/entity/Q395212', 'aLabel': 'agogô', 'b': 'http://www.wikidata.org/entity/Q101401', 'bLabel': 'campana'}
http://www.wikidata.org/entity/Q395212
Q395212


In [8]:
## Creamos los tres diccionarios pedidos, teniendo en cuenta que a es subconcepto directo de b
etiqueta = dict()
subclase = dict()
superclase = dict()
for elem in data :
    etiqueta [elem ['a'] [31:]] = elem ['aLabel']
    etiqueta [elem ['b'] [31:]] = elem ['bLabel']
    ## Si todavia no se había añadido se crea una lista de un elemento, si no se inserta a la lista previa
    if (subclase.get(elem ['b'] [31:]) != None):
        subclase [elem ['b'] [31:]].append(elem ['a'] [31:])
    else:
        subclase [elem ['b'] [31:]] = [elem ['a'] [31:]]
    if (superclase.get(elem ['a'] [31:])!=None):
        superclase [elem ['a'] [31:]].append(elem ['b'] [31:])
    else:
        superclase [elem ['a'] [31:]] = [elem ['b'] [31:]]

In [9]:
print (etiqueta['Q34379'])
print (subclase.get('Q17172850'))
print (superclase.get('Q5994'))

instrumento musical
['Q27911', 'Q31687', 'Q37137', 'Q186506', 'Q210970', 'Q223166', 'Q1067089', 'Q6164901', 'Q6983813', 'Q25303818', 'Q54285279', 'Q54285818', 'Q60396389', 'Q98116969', 'Q101436564']
['Q3152898', 'Q4951628', 'Q52954']


### 3) Imprimir un subárbol de la taxonomía

Crea una operación _print_tree_ que imprimir la jerarquía de clases a partir de un concepto y hasta un nivel de profundidad determinado.

Por ejemplo, a continuación podemos ver el principio de la jerarquía de [voces](https://www.wikidata.org/wiki/Q17172850) con 3 niveles de profundidad:

```
0 voz (Q17172850)
  1 operatic vocal (Q101436564)
  1 alto (Q6983813)
   2 mezzosoprano ligera (Q6012300)
   2 boy alto (Q53395277)
   2 alto castrato (Q53395016)
   2 contralto (Q37137)
  1 contralto (Q37137)
   2 contralto cómica (Q5785182)
   2 lyric contralto (Q54635214)
   2 Tenorino (Q6141663)
   2 contralto de coloratura (Q54635184)
   2 deep contralto (Q54635335)
   2 contralto dramática (Q5785183)
  1 bajo (Q27911)
   2 heavy acting bass (Q54636271)
   2 bajo profundo (Q2532487)
   2 bajo buffo (Q1002146)
   ...
```

Como ocurre en todas las grandes bases de conocimiento, dentro de Wikidata hay información que no ha sido bien introducida o está mal clasificada. ¿Puedes encontrar algún ejemplo concreto dentro de la jerarquía de instrumentos?

In [10]:
## Esquema de muestra
print ('0 '+etiqueta['Q17172850']+ ' (' + 'Q17172850' + ')')

0 voz (Q17172850)


In [11]:
## Definimos la funcion que para una etiqueta dada muestra 2 niveles de profundidad haciendo uso del diccionario subclase
def print_tree(etiq):
    print ('0 '+etiqueta[etiq]+ ' (' + etiq + ')')
    if subclase.get(etiq)!=None:
        for uno in subclase.get(etiq):
            print ('  1 '+etiqueta[uno]+ ' (' + uno + ')')
            if subclase.get(uno)!=None:
                for dos in subclase.get(uno):
                    print ('    2 '+etiqueta[dos]+ ' (' + dos + ')')

print_tree('Q17172850')

0 voz (Q17172850)
  1 bajo (Q27911)
    2 bajo-barítono (Q810480)
    2 bajo buffo (Q1002146)
    2 bajo profundo (Q2532487)
    2 bajo cantante (Q3636053)
    2 grave (Q5885030)
    2 octavist (Q7082656)
    2 bajo caractère (Q20638448)
    2 high bass (Q54636007)
    2 dramatic high bass (Q54636036)
    2 serious bass (Q54636068)
    2 heavy acting bass (Q54636271)
    2 Bass bourdon (Q64363543)
  1 barítono (Q31687)
    2 bajo-barítono (Q810480)
    2 character baritone (Q1062931)
    2 heldenbaritone (Q1601737)
    2 Barítono buffo (Q5721499)
    2 Barítono ligero (Q5721503)
    2 grave (Q5885030)
    2 Barítono lírico (Q8243255)
    2 Barítono dramático (Q8243257)
    2 baryton-noble (Q19740895)
    2 barítono Martin (Q21478751)
    2 Barítono martín (Q25404193)
    2 barítono verdiano (Q54635681)
    2 acting baritone (Q54635751)
    2 Kavalierbariton (Q54635784)
  1 contralto (Q37137)
    2 contralto cómica (Q5785182)
    2 contralto dramática (Q5785183)
    2 Tenorino (Q6141663

### 4) Obtener los LCS

Crea una operación _lcs_ que devuelva todos los LCS de dos conceptos determinados. Recuerda que un concepto C es LCS(A, B) si es más general que ambos y no se puede especializar más sin dejar de serlo.

Para implementarlo seguramente te resulte útil tener otro método que devuelva todos los conceptos más generales que uno dado. _Pista: es fácil de implementar usando operaciones entre conjuntos_. 

Ejemplos:

```
mezzosoprano dramática (Q6012297), mezzosoprano ligera (Q54634726), mezzosoprano (Q186506)
LCS('Q6012297', 'Q54634726') = {'Q186506'}

grave (Q5885030), mezzosoprano ligera (Q6012300), voz (Q17172850)
LCS('Q5885030', 'Q6012300') = {'Q17172850'}

tenor (Q27914)
LCS('Q27914', 'Q27914') = {'Q27914'}

viola eléctrica (Q15336282), bajo eléctrico (Q64166304), instrumento de cuerda (Q1798603), electrófono (Q105738), necked box lutes (Q55724840)
LCS('Q15336282', 'Q64166304') = {'Q55724840', 'Q105738', 'Q1798603}
```

In [12]:
## La funcion antecesores construye recursivamente por union de conjuntos todos los elementos de los que es subconcepto
def antecesores(etiq):
    conjunto = {etiq}
    if superclase.get(etiq)!=None:
        for elem in superclase.get(etiq):
            conjunto = conjunto | antecesores(elem)
    return conjunto

In [13]:
# Obtenemos LCS haciendo interseccion de los antecesores de a y b, quedandonos con aquellos que ocupen las posiciones inferiores
# Esto se constata cuando ninguna de sus subclases directas (diccionario) cumplen ser antecesor comun de a y b
def lcs(etiq_a, etiq_b):
    anteriores = antecesores(etiq_a) & antecesores(etiq_b)
    conjunto = set()
    for elem in anteriores:
        valido = True
        if subclase.get(elem)!=None:
            for x in subclase.get(elem):
                valido = valido and not(x in anteriores)
        if valido:
            conjunto.add(elem)
    
    return conjunto

In [14]:
print(lcs('Q6012297', 'Q54634726'))
print(lcs('Q5885030', 'Q6012300'))
print(lcs('Q27914', 'Q27914'))
print(lcs('Q15336282', 'Q64166304'))

{'Q186506'}
{'Q17172850'}
{'Q27914'}
{'Q105738', 'Q55724840', 'Q1798603'}


### 5) Obtener caminos mínimos

Crea una operación _path_ que calcule el camino mínimo entre dos conceptos A y B siendo A más o igual de general que B. Como la taxonomía no tiene ciclos puedes implementarlo como una búsqueda en profunidad. Ten en cuenta que los caminos sólo pueden contener conceptos más específicos o iguales a A y más generales o iguales a B.

Ejemplos:

```
path('Q186506', 'Q54634726') = [mezzosoprano (Q186506), mezzosoprano ligera (Q54634726)]

path('Q17172850', 'Q6012300') = [voz (Q17172850), alto (Q6983813), mezzosoprano ligera (Q6012300)]

path('Q27914', 'Q27914') = [tenor (Q27914)]

path('Q34379', 'Q55724840') = [instrumento musical (Q34379), cordófono (Q1051772), composite chordophones (Q19588495), lutes (Q1808578), handle lutes (Q30038759), necked lutes (Q55724833), necked box lutes (Q55724840)]
 ```

In [15]:
## Definimos la funcion sucesores analoga a antecesores, por union de conjuntos
def sucesores(etiq):
    conjunto = {etiq}
    if subclase.get(etiq)!=None:
        for elem in subclase.get(etiq):
            conjunto = conjunto | sucesores(elem)
    return conjunto

In [16]:
import sys
## La funcion path tiene como precondicion que a es subconcepto de b
## Haciendo la interseccion de los antecesores de a y los sucesores de b obtenemos los elementos que pueden estar 
## en el camino de b hacia a
## Recursivamente partiendo de b obtenemos el camino de b hacia a como b concatenado al minimo de los caminos de las subclases
## directas de b (diccionario) hacia a

def path (etiq_a,etiq_b):
    conjunto = sucesores(etiq_b) and antecesores(etiq_a)
    lista = [etiqueta[etiq_b]+ ' (' + etiq_b + ')']
    if subclase.get(etiq_b)!=None:
        aux=[]
        longitud=sys.maxsize
        for elem in subclase.get(etiq_b):
            if (elem in conjunto):
                camino=path(etiq_a,elem)
                if(len(camino)<longitud):
                    aux=camino
                    longitud=len(aux)
        lista.extend(aux)
    return lista

## Camino_min obtiene el camino minimo entre dos conceptos haciendo uso de la funcion path, y ordenando los argumentos en
## funcion de cual es el mas cercano a la raíz
def camino_min(etiq_a,etiq_b):
    if (etiq_a == etiq_b):
        return [etiqueta[etiq_a]+ ' (' + etiq_a + ')']
    elif (etiq_a in sucesores(etiq_b)):
        return path(etiq_a, etiq_b)
    elif (etiq_b in sucesores(etiq_a)):
        return path(etiq_b, etiq_a)
    else:
        return None

In [17]:
## Pruebas de depuracion para comprobar que funciona correctamente
print(antecesores('Q17172850'))
print(path('Q186506','Q34379'))
print(path('Q17172850','Q34379'))
print(path('Q98329515','Q34379'))
print(path('Q186506','Q98329515'))

{'Q17172850', 'Q34379', 'Q98329515'}
['instrumento musical (Q34379)', 'voz (Q17172850)', 'mezzosoprano (Q186506)']
['instrumento musical (Q34379)', 'voz (Q17172850)']
['instrumento musical (Q34379)', 'Instrumento de tono continuo (Q98329515)']
['Instrumento de tono continuo (Q98329515)', 'voz (Q17172850)', 'mezzosoprano (Q186506)']


In [18]:
print(camino_min('Q186506', 'Q54634726'))
print(camino_min('Q17172850', 'Q6012300'))
print(camino_min('Q27914', 'Q27914'))
print(camino_min('Q34379', 'Q105738'))
print(camino_min('Q34379', 'Q55724840'))

['mezzosoprano (Q186506)', 'mezzosoprano ligera (Q54634726)']
['voz (Q17172850)', 'alto (Q6983813)', 'mezzosoprano ligera (Q6012300)']
['tenor (Q27914)']
['instrumento musical (Q34379)', 'electrófono (Q105738)']
['instrumento musical (Q34379)', 'cordófono (Q1051772)', 'composite chordophones (Q19588495)', 'lutes (Q1808578)', 'handle lutes (Q30038759)', 'necked lutes (Q55724833)', 'necked box lutes (Q55724840)']


### 6) Calcular la similitud

Implementa una operación _similarity_ que calcule la similtud entre dos conceptos. Debe devolver tanto el valor númerico de similitud como los caminos desde la raiz al LCS y desde el LCS a cada uno de los dos conceptos.

Ten en cuenta que debes usar un LCS que maximice el valor de similitud. Si la información de Wikidata no ha cambiado, los valores de similitud deberían coincidir con los que aparecen en los ejemplos pero los caminos no tienen por qué. Y recuerda que no es lo mismo el números de aristas de un camino que el número de nodos del camino.

Ejemplos:

```
similarity('Q6012297', 'Q54634726')
0.5
[instrumento musical (Q34379), voz (Q17172850), mezzosoprano (Q186506)]
[mezzosoprano (Q186506), mezzosoprano dramática (Q6012297)]
[mezzosoprano (Q186506), mezzosoprano ligera (Q54634726)]

similarity('Q186506', 'Q54634726')
0.6666666666666666
[instrumento musical (Q34379), voz (Q17172850), mezzosoprano (Q186506)]
[mezzosoprano (Q186506)]
[mezzosoprano (Q186506), mezzosoprano ligera (Q54634726)]

similarity('Q27914', 'Q27914')
1.0
[instrumento musical (Q34379), voz (Q17172850), high voice (Q98116969), tenor (Q27914)]
[tenor (Q27914)]
[tenor (Q27914)]

similarity('Q76239', 'Q78987')
0.42857142857142855
[instrumento musical (Q34379), cordófono (Q1051772), instrumento de cuerda (Q1798603), instrumento de cuerda pulsada (Q230262)]
[instrumento de cuerda pulsada (Q230262), cítara (Q76239)]
[instrumento de cuerda pulsada (Q230262), plucked necked box lutes (Q57306162), guitarra (Q6607), guitarra eléctrica (Q78987)]
```

In [19]:
## Obtenemos la similaridad por la formula siguiente: 𝑆𝑖𝑚(𝐴,𝐵)= 𝛿(𝑟𝑜𝑜𝑡,𝐶) / (𝛿(𝑟𝑜𝑜𝑡,𝐶)+𝛿(𝐶,𝐴)+𝛿(𝐶,𝐵))
## Devuelve la tupla (𝑆𝑖𝑚(𝐴,𝐵) , camino(𝑟𝑜𝑜𝑡,𝐶), camino(𝐶,𝐴) , camino(𝐶,𝐵)) donde c= LCS(A,B)
def similaridad(etiq_a,etiq_b):
    c = lcs(etiq_a, etiq_b).pop()
    raiz_c=camino_min('Q34379', c)
    c_a =camino_min(c, etiq_a)
    c_b =camino_min(c, etiq_b)
    sim= (len(raiz_c)-1) / (len(raiz_c)-1+len(c_a)-1+len(c_b)-1)
    return [sim,raiz_c,c_a,c_b]

In [20]:
## Ejemplos de comprobacion
a=similaridad('Q6012297', 'Q54634726')
b=similaridad('Q186506', 'Q54634726')
c=similaridad('Q27914', 'Q27914')
d=similaridad('Q76239', 'Q78987')
for i in range(4):
    print(a[i])
for i in range(4):
    print(b[i])
for i in range(4):
    print(c[i])
for i in range(4):
    print(d[i])

0.5
['instrumento musical (Q34379)', 'voz (Q17172850)', 'mezzosoprano (Q186506)']
['mezzosoprano (Q186506)', 'mezzosoprano dramática (Q6012297)']
['mezzosoprano (Q186506)', 'mezzosoprano ligera (Q54634726)']
0.6666666666666666
['instrumento musical (Q34379)', 'voz (Q17172850)', 'mezzosoprano (Q186506)']
['mezzosoprano (Q186506)']
['mezzosoprano (Q186506)', 'mezzosoprano ligera (Q54634726)']
1.0
['instrumento musical (Q34379)', 'voz (Q17172850)', 'high voice (Q98116969)', 'tenor (Q27914)']
['tenor (Q27914)']
['tenor (Q27914)']
0.42857142857142855
['instrumento musical (Q34379)', 'cordófono (Q1051772)', 'instrumento de cuerda (Q1798603)', 'instrumento de cuerda pulsada (Q230262)']
['instrumento de cuerda pulsada (Q230262)', 'cítara (Q76239)']
['instrumento de cuerda pulsada (Q230262)', 'plucked necked box lutes (Q57306162)', 'guitarra (Q6607)', 'guitarra eléctrica (Q78987)']


### 7) Análisis de las similitudes

Calcula la similitud 2 a 2 de los siguientes instrumentos y explica razonadamente si los valores obtenidos tienen sentido de acuerdo a tu intuición sobre si se parecen o no.

```
piano (Q5994), guitarra (Q6607), guitarra eléctrica (Q78987), flauta (Q11405), trompeta (Q8338)
```

In [21]:
## Muestra de las similaridades dos a dos para el posterior comentario
print(similaridad('Q5994', 'Q6607')[0],'piano','guitarra') 
print(similaridad('Q5994', 'Q78987')[0],'piano','guitarra electrica')
print(similaridad('Q5994', 'Q11405')[0],'piano','flauta')
print(similaridad('Q5994', 'Q8338')[0],'piano','trompeta') 
print(similaridad('Q6607', 'Q78987')[0],'guitarra','guitarra electrica')
print(similaridad('Q6607', 'Q11405')[0],'guitarra','flauta') 
print(similaridad('Q6607', 'Q8338')[0],'guitarra','trompeta')
print(similaridad('Q78987', 'Q11405')[0],'guitarra eléctrica','flauta')
print(similaridad('Q78987', 'Q8338')[0],'guitarra eléctrica','trompeta')
print(similaridad('Q11405', 'Q8338')[0],'flauta','trompeta')

0.2727272727272727 piano guitarra
0.25 piano guitarra electrica
0.0 piano flauta
0.0 piano trompeta
0.8333333333333334 guitarra guitarra electrica
0.0 guitarra flauta
0.0 guitarra trompeta
0.0 guitarra eléctrica flauta
0.0 guitarra eléctrica trompeta
0.4 flauta trompeta


### COMENTARIOS
#### Todo va de acuerdo al sentido común. Existe relacion entre instrumentos del mismo tipo, que recordamos son viento,cuerda, percusion. Además entre instrumentos de distinto tipo la similitud es 0.0 pues su LCS es la raíz de la taxonomía, instrumento musical.
#### Podemos añadir que dentro de los intrumentos de cuerda hay gran similitud entre los dos tipos de guitarra, la normla y la electrica (0.83), y tambien es lógico que el piano tenga mayor similitud con la guitarra clasica que con la electrica